In [8]:
def getWords(text: String): Array[String] = {
    //FILL THIS IN
    text.split(" ").map(a=>a.toArray.filter(b=>b.isLetter)).map(c=>c.mkString)
}

val s = Array("Jack, the man\nWoolly the mammoth!","Jack, the man\nWoolly the mammoth!")
val s_p = sc.parallelize(s)
s_p.map(rdd=>getWords(rdd.toLowerCase())).collect()

<console>: 36: error: not enough arguments for method collect: (pf: PartialFunction[String,B])(implicit bf: scala.collection.generic.CanBuildFrom[Array[String],B,That])That.

In [12]:
val filters = Array("fifa", "qatar", "soccer")
val contains_example = "The world cup fifa is in Qatar this year".exists(c=>c.contains(filters)
                                                                        )

<console>: 25: error: value contains is not a member of Char

In [31]:
val contains_example = "world\ncup".contains("world")
val exists_example_true = Array(1,3,5,7,8,13).exists(v => v%2==0)
val exists_example_false = Array(1,3,5,7,9,13).exists(v => v%2==0)

contains_example: Boolean = true
exists_example_true: Boolean = true
exists_example_false: Boolean = false


In [15]:
"The world cup is in Qatar this year".exists(v => v=='c')

res9: Boolean = true


In [34]:
val filters = Array("cup","football")
// val keys = "cup football"
val a = Array("I watch world Cup","I watch world","world\ncup")
val a_p = sc.parallelize(a)
a_p.filter(b=>filters.exists(filter => b.toLowerCase().contains(filter))).collect()
// filters.exists(filter => a.contains(filter))

filters: Array[String] = Array(cup, football)
a: Array[String] =
Array(I watch world Cup, I watch world, world
cup)
a_p: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[14] at parallelize at <console>:30
res24: Array[String] =
Array(I watch world Cup, world
cup)


In [36]:
"world\ncup".toLowerCase().contains("cup")

res26: Boolean = true


In [7]:
import org.apache.spark.streaming.{Seconds, StreamingContext}
val ssc = new StreamingContext(sc, Seconds(10))
val lines = ssc.socketTextStream("localhost", 4444)
ssc.checkpoint("checkpoint")
val words = lines.flatMap(l => l.split(" "))

//Total words in the microbatch (10 seconds)
//This will print every 10 seconds


//Total number of words in every 30 second window sliding every 20 seconds
//The print is at each point that the window slides (20, 40, 60 ,80,....)
//This will print at 20 (total words in 20 seconds since this is the first slide)
//at 40 seconds (total words in the last 3 10 second intervals)
//at 60 seconds, etc.
val wcount = words.window(Seconds(30),Seconds(20))
wcount.foreachRDD((rdd,time) => print(time.toString.takeRight(10),": Total  ",rdd.collect()(1)))

import org.apache.spark.streaming.{Seconds, StreamingContext}
ssc: org.apache.spark.streaming.StreamingContext = org.apache.spark.streaming.StreamingContext@3a847789
lines: org.apache.spark.streaming.dstream.ReceiverInputDStream[String] = org.apache.spark.streaming.dstream.SocketInputDStream@7637d6fe
words: org.apache.spark.streaming.dstream.DStream[String] = org.apache.spark.streaming.dstream.FlatMappedDStream@24307ba3
wcount: org.apache.spark.streaming.dstream.DStream[String] = org.apache.spark.streaming.dstream.WindowedDStream@91edb87


In [54]:
import org.apache.spark.streaming.{Seconds, StreamingContext}
import org.apache.spark.rdd.PairRDDFunctions._
import scala.io.Source
import scala.collection.mutable.ArrayBuffer
sc.setLogLevel("ERROR")

val filters = Source.fromFile("fifa2022_words.txt").getLines.toArray

val sentimentFilePath = "AFINN-111.txt"

val wordSentiments = sc.textFile(sentimentFilePath).map { line => 
    (line.split("\t")(0),line.split("\t")(1).toInt)
}.cache()

def getWords(text: String): Array[String] = {
    text.split(" ").map(a=>a.toArray.filter(b=>b.isLetter)).map(c=>c.mkString)
}

val ssc = new StreamingContext(sc,Seconds(10.toLong))
val text = ssc.socketTextStream("localhost", 4444)

//filter tweets only keeping world cup related ones
//Use transform to work on each rdd (text is a DStream object, not an RDD)
//filter using exists and contains
//Also, convert the text to lowercase (all the keywords are in lowercase)
val filteredText = text.transform(
                    rdd => rdd.filter( //rdd
                        a => filters.exists( //element
                            key => a.toLowerCase().contains(key))))

//convert all tweets in filteredText into a single array of words (think flatMap)
val words = filteredText.flatMap(getWords(_))

//get sentiments
//we need to convert each word into a pair (word, 1) to count the number of words
//apply transform to join each rdd with the wordSentiment rdd using fullOuterJoin
//Use match to convert Option to a new paired rdd (count, 1*sentiment)
val sentiment = words.map(word => (word, 1))
                  .transform{
                      rdd => rdd.fullOuterJoin(wordSentiments)
                          .flatMap(pair => pair match {
                                    case (word,(Some(count),None)) => Some(count,None)
                                    case (word,(None,senti)) => None
                                    case (word,(Some(count),Some(senti))) => Some(count,1*senti)
                                    // case (_) => None
                                    }
                              )
                            }
// val b = sentiment.flatten


<console>: 155: error: value flatten is not a member of org.apache.spark.streaming.dstream.DStream[(Int, Any)]

In [39]:
sc.setLogLevel("ERROR")
// words.print()
sentiment.print()
ssc.start

-------------------------------------------
Time: 1670109820000 ms
-------------------------------------------

-------------------------------------------
Time: 1670109830000 ms
-------------------------------------------
(1,-2)
(1,None)
(1,None)



In [40]:
ssc.stop(false)

22/12/03 18:23:52 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver


In [31]:
val t = "1670107490000 ms"
t.dropRight(3).takeRight(7)

t: String = 1670107490000 ms
res19: String = 7490000


In [33]:
average(1.2,1.2)

<console>: 85: error: not found: value average

In [55]:
val a = sc.parallelize(Array(Some(1,-2),Some(1,None),Some(1,None)))

a: org.apache.spark.rdd.RDD[Some[(Int, Any)]] = ParallelCollectionRDD[173] at parallelize at <console>:96


In [57]:
a.flatMap()

<console>: 98: error: Cannot prove that (Int, Any) <:< Option[B].

In [48]:
val b = a.flatten

b: Array[(Int, Any)] = Array((1,-2), (1,None), (1,None))


In [50]:
b(1)._2

res32: Any = None


In [51]:
a(1)._2

<console>: 98: error: value _2 is not a member of Some[(Int, Any)]

In [19]:
val p = Array((0.0,-2.0),(0.0,0.0),(0.0,0.0))
val a = sc.parallelize(p)
a.map(t=>t._2).reduce((a,b) => a+b)

p: Array[(Double, Double)] = Array((0.0,-2.0), (0.0,0.0), (0.0,0.0))
a: org.apache.spark.rdd.RDD[(Double, Double)] = ParallelCollectionRDD[5] at parallelize at <console>:27
res11: Double = -2.0
